In [1]:
import requests
from bs4 import BeautifulSoup
import os
import json
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer
from langchain.document_loaders import PyPDFLoader, CSVLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.chains.question_answering import load_qa_chain
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_groq import ChatGroq
from dotenv import load_dotenv
load_dotenv()

def get_data_from_website(url):
    # Get response from the server
    response = requests.get(url)
    if response.status_code == 500:
        print("Server error")
        return
    
    # Parse the HTML content using BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Step 1: Find all tab titles
    tab_titles = soup.find_all("div", class_="elementor-tab-title")

    # Step 2: Find all corresponding tab contents
    tab_data = {}

    for title_div in tab_titles:
        tab_id = title_div.get("data-tab")
        tab_title = title_div.get_text(strip=True)

        matching_content = soup.find("div", class_="elementor-tab-content", attrs={"data-tab": tab_id})
        tab_content = matching_content.get_text(separator="\n", strip=True) if matching_content else ""

        tab_data[tab_title] = tab_content

    # ✅ Create 'data' folder if it doesn't exist
    os.makedirs("data", exist_ok=True)

    # ✅ Save as JSON file
    with open("data/tab_data.json", "w", encoding="utf-8") as f:
        json.dump(tab_data, f, ensure_ascii=False, indent=2)

    print("Data saved to data/tab_data.json")


c:\chtn\gen_ai\hitesh\project_xa001\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
get_data_from_website("https://www.dinecollege.edu/academics/academic-policies/")

Data saved to data/tab_data.json


In [3]:
def append_ferpa_data(url, output_filename="data//tab_data.json"):
    """
    Fetches data from a FERPA-related website, processes it, and appends it to a JSON file.

    Args:
        url (str): The URL of the website to scrape.
        output_filename (str, optional): The name of the JSON file to save/append data to.
            Defaults to "data//tab_data.json".
    """
    try:
        response = requests.get(url)
        response.raise_for_status()
    except requests.exceptions.RequestException as e:
        print(f"Error fetching URL: {e}")
        return

    soup = BeautifulSoup(response.content, 'html.parser')

    data = {}
    h3_tags = soup.find_all('h3')
    for h3_tag in h3_tags:
        question = h3_tag.text.strip()
        answer_parts = []
        sibling = h3_tag.find_next_sibling()
        while sibling and sibling.name in ['p', 'div', 'ul', 'ol']:
            answer_parts.append(sibling.text.strip())
            sibling = sibling.find_next_sibling()
        answer = " ".join(answer_parts).strip()
        if question and answer:
            data[question] = answer

    modified_data = {}
    for key, value in data.items():
        new_key = key.rstrip(":")
        modified_value = value.replace("Back to Top", "").strip()
        if len(modified_value.split()) >= 5:
            modified_data[new_key] = modified_value

    _append_to_json(modified_data, output_filename)

def append_civil_rights_data(url, output_filename="data//tab_data.json"):
    """
    Fetches data from a civil rights laws website, processes it, and appends it to a JSON file.

    Args:
        url (str): The URL of the website to scrape.
        output_filename (str, optional): The name of the JSON file to save/append data to.
            Defaults to "data//tab_data.json".
    """
    try:
        response = requests.get(url)
        response.raise_for_status()
    except requests.exceptions.RequestException as e:
        print(f"Error fetching URL: {e}")
        return

    soup = BeautifulSoup(response.content, 'html.parser')

    final_data = {}

    # Extract the main heading and description
    main_heading = soup.find('h1', class_='usa-hero__heading')
    main_desc = soup.find('div', class_='field--name-body')

    if main_heading and main_desc:
        title = main_heading.text.strip()
        description = main_desc.text.strip()
        final_data[title] = description

    # Now extract the cards information
    cards = soup.find_all('div', class_='card-image-top-txt')

    for card in cards:
        card_title_tag = card.find('div', class_='field--name-field-ed-card-image-top-title')
        card_summary_tag = card.find('div', class_='field--name-field-ed-card-image-top-summary')
        card_link_tag = card.find('div', class_='field--name-field-ed-card-image-top-link')

        if card_title_tag and card_summary_tag:
            card_title = card_title_tag.text.strip()
            card_summary = card_summary_tag.text.strip()

            # Get the link if available
            link = ""
            if card_link_tag and card_link_tag.find('a'):
                href = card_link_tag.find('a')['href']
                if href.startswith("/"):
                    href = "https://www.ed.gov" + href
                link = href
            final_data[card_title] = f"{card_summary} link :- {link}".strip()

    _append_to_json(final_data, output_filename)

def append_file_complaint_data(url, output_filename="data//tab_data.json"):
    """
    Fetches data from the file a complaint website, processes it, and appends it to a JSON file.

    Args:
        url (str): The URL of the website to scrape.
        output_filename (str, optional): The name of the JSON file to save/append data to.
            Defaults to "data//tab_data.json".
    """
    try:
        response = requests.get(url)
        response.raise_for_status()
    except requests.exceptions.RequestException as e:
        print(f"Error fetching URL: {e}")
        return

    soup = BeautifulSoup(response.content, 'html.parser')

    # Remove unnecessary tags
    for tag in soup(["script", "style", "footer", "nav", "header", "aside"]):
        tag.decompose()

    # Remove known banners or sections
    for div in soup.find_all(['div', 'section'], class_=[
        'usa-banner', 'header', 'navigation', 'menu', 'site-header',
        'usa-footer', 'main-header', 'branding', 'footer-links'
    ]):
        div.decompose()

    for elem in soup.find_all(id=[
        'header', 'footer', 'navbar', 'skip-link', 'back-to-top'
    ]):
        elem.decompose()

    # Get heading
    heading = soup.find('h1')
    key = heading.get_text(strip=True) if heading else "No Heading Found"

    # Collect all visible text
    body_text = soup.get_text(separator='\n')
    lines = [line.strip() for line in body_text.splitlines() if line.strip()]

    # Keywords/phrases to exclude
    unwanted_keywords = [
        "Complaint Forms", "Electronic Complaint Form Learn how to file", "How OCR Evaluates Complaints",
        "FAQs on the Complaint Process", "Customer Service Standards for the Case Resolution Process",
        "Complainant and Interviewee Rights and Protections", "Rights and protections",
        "Office of Communications and Outreach", "Page Last Reviewed"
    ]

    # Remove lines matching unwanted sections
    filtered_lines = [
        line for line in lines
        if not any(keyword.lower() in line.lower() for keyword in unwanted_keywords)
    ]

    # Try to add Electronic Complaint Form and Fillable PDF Complaint Form links
    extra_links_text = ""
    electronic_form = soup.find('a', string=lambda text: text and 'Electronic Complaint Form' in text)
    pdf_form = soup.find('a', string=lambda text: text and 'Fillable PDF Complaint Form' in text)

    if electronic_form:
        href = electronic_form.get('href')
        extra_links_text += f"\nElectronic Complaint Form: {href}"
    if pdf_form:
        href = pdf_form.get('href')
        extra_links_text += f"\nFillable PDF Complaint Form: {href}"

    # Final value
    value = ' '.join(filtered_lines) + extra_links_text

    # Result dict
    result = {key: value}

    _append_to_json(result, output_filename)

def _append_to_json(new_data, output_filename):
    """
    Appends a dictionary of data to an existing JSON file or creates a new one.

    Args:
        new_data (dict): The dictionary data to append.
        output_filename (str): The name of the JSON file.
    """
    try:
        with open(output_filename, 'r+', encoding='utf-8') as f:
            try:
                existing_data = json.load(f)
                existing_data.update(new_data)
                f.seek(0)
                json.dump(existing_data, f, ensure_ascii=False, indent=4)
                f.truncate() # Remove remaining part if new data is shorter
            except json.JSONDecodeError:
                print("Error decoding existing JSON file. Overwriting with new data.")
                f.seek(0)
                json.dump(new_data, f, ensure_ascii=False, indent=4)
                f.truncate()
    except FileNotFoundError:
        with open(output_filename, 'w', encoding='utf-8') as f:
            json.dump(new_data, f, ensure_ascii=False, indent=4)

In [4]:
ferpa_url = "https://studentprivacy.ed.gov/ferpa"
civil_rights_url = "https://www.ed.gov/laws-and-policy/civil-rights-laws"
file_complaint_url = "https://www.ed.gov/laws-and-policy/civil-rights-laws/file-complaint"

append_ferpa_data(ferpa_url)
append_civil_rights_data(civil_rights_url)
append_file_complaint_data(file_complaint_url)

print("Data appended to data//tab_data.json")

Data appended to data//tab_data.json


In [5]:
# Load JSON data
with open('data/tab_data.json', 'r', encoding='utf-8') as f:
    tab_data = json.load(f)

# Combine tab title and content into a document
documents = [f"{key}: {value}" for key, value in tab_data.items()]
metadata = list(tab_data.keys())

# Load a pre-trained embedding model from Hugging Face
model = SentenceTransformer('all-MiniLM-L6-v2')

# Generate embeddings
embeddings = model.encode(documents, convert_to_numpy=True, show_progress_bar=True)

# Create FAISS index
embedding_dim = embeddings.shape[1]
index = faiss.IndexFlatL2(embedding_dim)  # Using L2 similarity
index.add(embeddings)

# Save the index
faiss.write_index(index, 'data/faiss_index.idx')

# Save the metadata for reverse lookup
with open('data/faiss_metadata.json', 'w', encoding='utf-8') as f:
    json.dump(metadata, f, ensure_ascii=False, indent=2)

print("✅ FAISS index and metadata saved.")

Batches: 100%|██████████| 2/2 [00:00<00:00,  2.64it/s]

✅ FAISS index and metadata saved.


In [6]:
# Load the saved FAISS index and metadata
index = faiss.read_index('data/faiss_index.idx')
with open('data/faiss_metadata.json', 'r', encoding='utf-8') as f:
    metadata = json.load(f)

# Also load the tab_data to fetch full content for results
with open('data/tab_data.json', 'r', encoding='utf-8') as f:
    tab_data = json.load(f)

def search_query(user_query, top_k=2):
    # Convert query to embedding
    query_vector = model.encode([user_query], convert_to_numpy=True)

    # Perform similarity search
    distances, indices = index.search(query_vector, top_k)

    # Print top-k matches
    print(f"\n🔍 Top {top_k} Results for: '{user_query}'\n")
    for i, idx in enumerate(indices[0]):
        title = metadata[idx]
        content = tab_data[title]
        score = distances[0][i]
        print(f"{i+1}. Title: {title}")
        print(f"   Score: {score:.4f}")
        print(f"   Content:\n{content[:300]}{'...' if len(content) > 300 else ''}\n")

# Example
search_query("Want to know about the academic appeals.")


🔍 Top 2 Results for: 'Want to know about the academic appeals.'

1. Title: Grades
   Score: 0.9125
   Content:
General Grade Appeal
Grades are determined solely by the individual faculty who taught the course for the session(s) or the semester(s). A student who wishes to contest a grade must first attempt to resolve the matter with the course faculty.
If the matter cannot be resolved with the instructor, the...

2. Title: Academics
   Score: 1.0904
   Content:
Academic Appeals
Students placed on academic probation or suspension may appeal to the Academic Standards Committee by filing an appeal form with the Office of the Registrar. The student has the right to appeal any action affecting their academic status by obtaining the appropriate form from the Off...



In [7]:
llm = ChatGroq(
        model="Llama3-8b-8192",
        temperature=0,
        max_tokens=4192,
        timeout=30,
        max_retries=2,
    )

def generate_answer(user_query, retrieved_titles, tab_data):
    # Combine relevant tab content
    retrieved_docs = "\n\n".join([f"{title}: {tab_data[title]}" for title in retrieved_titles if title in tab_data])

    # Construct the prompt
    prompt = f"""
You are an expert assistant. Answer the user's question **only** based on the information provided below. 
Do **not** use any external knowledge or generate content beyond the provided documents.

If the answer is **not present** in the information, reply with: 
"I'm sorry, but that question is outside the scope of the provided information."

Information:
{retrieved_docs}

Question: {user_query}

Answer in a clear, helpful, and concise manner.
"""


    # Generate response using ChatGroq (LLaMA3)
    response = llm.invoke(prompt)
    return response


In [ ]:
def search_and_generate(user_query, top_k=3):
    query_vector = model.encode([user_query], convert_to_numpy=True)
    distances, indices = index.search(query_vector, top_k)

    retrieved_titles = [metadata[idx] for idx in indices[0]]

    # Generate the answer using LLaMA (ChatGroq)
    answer = generate_answer(user_query, retrieved_titles, tab_data)

    print(f"\n🧠 Answer:\n{answer.content}")


# question 1 :- Want to know about the academic appeals.
# question 2 :- How to File A Complaint for civil rights.
# question 3 :- Can you tell me the capital of france?
search_and_generate("Can you tell me the capital of france?")



🧠 Answer:
I'm sorry, but that question is outside the scope of the provided information.
